In [99]:
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
import torch
from sentence_transformers import SentenceTransformer

# Apply tqdm to all .apply() functions by using progress_apply
tqdm.pandas()

In [100]:
# Use GPU if available
"""
Reference: https://pytorch.org/get-started/locally/
"""

# Check for NVIDIA GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use CUDA (NVIDIA GPU)
    print("Using NVIDIA GPU (CUDA)")

# Check for Mac Silicon GPU (MPS)
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Metal Performance Shaders (Mac Silicon GPU)
    print("Using Mac GPU (MPS)")

# Default to CPU if no GPU is available
else:
    device = torch.device("cpu")
    print("Using CPU")

Using Mac GPU (MPS)


In [101]:
# Reading the dataset
alexator_df = pd.read_csv('../data/alexator_stickers_desc.csv')
flaticon_df = pd.read_csv('../data/flaticon_stickers_desc.csv')
freepik_df = pd.read_csv('../data/freepik_stickers_desc.csv')

In [102]:
alexator_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4331 entries, 0 to 4330
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     4331 non-null   object
 1   description  4331 non-null   object
dtypes: object(2)
memory usage: 67.8+ KB


In [103]:
flaticon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30402 entries, 0 to 30401
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        30402 non-null  object
 1   title        30402 non-null  object
 2   tags         30402 non-null  object
 3   filename     30402 non-null  object
 4   description  30402 non-null  object
dtypes: object(5)
memory usage: 1.2+ MB


In [104]:
freepik_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9844 entries, 0 to 9843
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        0 non-null      float64
 1   tags         9814 non-null   object 
 2   image        9844 non-null   object 
 3   filename     9844 non-null   object 
 4   description  9844 non-null   object 
dtypes: float64(1), object(4)
memory usage: 384.7+ KB


In [105]:
# Concatenate the DataFrames row-wise
merged_df = pd.concat([alexator_df, flaticon_df, freepik_df], ignore_index=True)

# Display the concatenated DataFrame
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44577 entries, 0 to 44576
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     44577 non-null  object
 1   description  44577 non-null  object
 2   image        40246 non-null  object
 3   title        30402 non-null  object
 4   tags         40216 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


## Handling duplicates in description and tags combination

In [106]:
# Replace duplicate descriptions with Null (Because most of them don't make sense and are incorrect)
duplicate_descriptions = merged_df['description'].duplicated(keep=False)
merged_df.loc[duplicate_descriptions, 'description'] = None

"""Some generated descriptions have errors such as "a big boy boy boy boy boy...."
So, we want to remove such descriptions that have incorrect descriptions
"""

"""
Reference: ChatGPT-4o
Prompt:  I need to check for a certain type of error and remove those rows in my dataframe. The error is that in some description, certain words are repeated multiple time consecutively. Example: "happy man with two polar polar polar polar polar polar polar polar polar polar polar polar polar polar polar polar polar"
"""
# Replace descriptions with repeated words with Null
def has_repeated_words(text):
    if not text or pd.isna(text):  # Skip null or empty strings
        return False
    
    # Regex to find words repeated consecutively at least 3 times (case insensitive)
    pattern = r'\b(\w+)(?:\s+\1){2,}\b'
    return bool(re.search(pattern, text, re.IGNORECASE))

merged_df.loc[merged_df['description'].apply(has_repeated_words), 'description'] = None

# Remove rows with exact same tags AND description, keeping the first occurrence (We don't want different immages with same details)
cleaned_df = merged_df.drop_duplicates(subset=['tags', 'description'], keep='first')

# Check the new number of rows
print(f"Number of rows after removing rows with exact same tags and descriptions: {cleaned_df.shape[0]}")

Number of rows after removing rows with exact same tags and descriptions: 38807


In [107]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38807 entries, 0 to 44576
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     38807 non-null  object
 1   description  26858 non-null  object
 2   image        35101 non-null  object
 3   title        27135 non-null  object
 4   tags         35091 non-null  object
dtypes: object(5)
memory usage: 1.8+ MB


## Handling Tags and Descriptions

In [108]:
def remove_duplicates(text):
    if not isinstance(text, str) or pd.isna(text) or text.strip().lower() == "nan":  
        return ""  # Return empty string for NaN or "nan" strings
    words = [word.strip() for word in text.split(",")]  # Split by commas and strip spaces
    unique_words = sorted(set(word.strip() for word in text.split(",")))
    return ', '.join(unique_words)  # Join back into a string

In [109]:
# Remove duplicates
final_df = cleaned_df.copy()
final_df["tags"] = final_df["tags"].progress_apply(remove_duplicates)

100%|██████████| 38807/38807 [00:00<00:00, 384300.03it/s]


In [110]:
### Need to conduct spelling correction in description column here as well

## Embedding using Sentence BERT

In [111]:
# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
sbert_model = sbert_model.to(device)

# Return SBERT embedding for a given text.
def embed_text(text):
    if pd.isna(text) or text.strip() == "":
        return np.zeros(384, dtype=np.float32)  # Return zero vector for missing values (SBERT output size = 384)
    return sbert_model.encode(text).astype(np.float32)

In [112]:
"""
Reference: https://sbert.net/
"""

# Apply SBERT embeddings to each columns
print("Embedding tags...")
final_df["tags_embedding"] = final_df["tags"].progress_apply(embed_text)

print("Embedding llm_description...")
final_df["description_embedding"] = final_df["description"].progress_apply(embed_text)

Embedding tags...


100%|██████████| 38807/38807 [06:11<00:00, 104.51it/s]


Embedding llm_description...


100%|██████████| 38807/38807 [04:41<00:00, 137.94it/s]


In [113]:
# Combining the embeddings
final_df["combined_embedding"] = final_df.apply(
    lambda row: np.concatenate([row["tags_embedding"], row["description_embedding"]]), axis=1
)

In [116]:
print(final_df["combined_embedding"].apply(lambda x: x.dtype).value_counts())

combined_embedding
float32    38807
Name: count, dtype: int64


## Linking Images

In [117]:
final_df.head()

,filename,description,image,title,tags,tags_embedding,description_embedding,combined_embedding
0,cartoonized_image_0.png,a red mailbox with a white background,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.056150388, 0.06784806, -0.03342953, 0.03755...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,cartoonized_image_1.png,an old car sticker,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.124234326, 0.07463955, -0.011985384, 0.004...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,cartoonized_image_10.png,None,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,cartoonized_image_100.png,a sticker with a cartoon pine wearing glasses,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.06495429, -0.04292717, 0.013164398, 0.0220...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,cartoonized_image_1000.png,a stained stained glass window with a humming,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.02791824, 0.07555939, 0.036227096, -0.01810...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [118]:
# Change file extensions from .png to .pt in the 'filename' column
final_df['filename'] = final_df['filename'].str.replace('.png', '.pt', regex=False)

# Display the first few rows to verify changes
final_df['filename'].head()

0       cartoonized_image_0.pt
1       cartoonized_image_1.pt
2      cartoonized_image_10.pt
3     cartoonized_image_100.pt
4    cartoonized_image_1000.pt
Name: filename, dtype: object

In [119]:
# Define the base directory and folders to search in
base_dir = "../data/tensor_images/"
folders = ["AlexatorStickers", "FlaticonStickers", "FreepikStickers"]

# Function to find the file path for a given filename
def find_image_path(filename):
    # Loop through each folder
    for folder in folders:
        # Construct full path
        full_path = os.path.join(base_dir, folder, filename)
        # Check if file exists at this path
        if os.path.exists(full_path):
            return full_path
    # Return None if file not found in any folder
    return None

# Create the new column by applying the function to the filename column
final_df['image_path'] = final_df['filename'].progress_apply(find_image_path)

# Display a sample of the DataFrame to verify
final_df.head()

100%|██████████| 38807/38807 [00:00<00:00, 44960.11it/s]


,filename,description,image,title,tags,tags_embedding,description_embedding,combined_embedding,image_path
0,cartoonized_image_0.pt,a red mailbox with a white background,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.056150388, 0.06784806, -0.03342953, 0.03755...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",../data/tensor_images/AlexatorStickers/cartoon...
1,cartoonized_image_1.pt,an old car sticker,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.124234326, 0.07463955, -0.011985384, 0.004...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",../data/tensor_images/AlexatorStickers/cartoon...
2,cartoonized_image_10.pt,None,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",../data/tensor_images/AlexatorStickers/cartoon...
3,cartoonized_image_100.pt,a sticker with a cartoon pine wearing glasses,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.06495429, -0.04292717, 0.013164398, 0.0220...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",../data/tensor_images/AlexatorStickers/cartoon...
4,cartoonized_image_1000.pt,a stained stained glass window with a humming,NaN,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.02791824, 0.07555939, 0.036227096, -0.01810...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",../data/tensor_images/AlexatorStickers/cartoon...


In [120]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38807 entries, 0 to 44576
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   filename               38807 non-null  object
 1   description            26858 non-null  object
 2   image                  35101 non-null  object
 3   title                  27135 non-null  object
 4   tags                   38807 non-null  object
 5   tags_embedding         38807 non-null  object
 6   description_embedding  38807 non-null  object
 7   combined_embedding     38807 non-null  object
 8   image_path             38807 non-null  object
dtypes: object(9)
memory usage: 3.0+ MB


In [121]:
final_df[['combined_embedding', 'image_path']].to_parquet('../data/processed_sticker_dataset.parquet', index=False)